In [1]:
import LeakDetector
import numpy as np
import pandas as pd
from os.path import isdir, join, basename
from datetime import datetime
from glob import glob

In [2]:
from device_ids import TV_ID_MAP
from log_analysis import get_crawl_parameter
from crawl_ids import CrawlRokuTop1KNoMITM
from df_utils import load_df
from nb_utils import make_latex_table

from ott_leaks import run_leak_detection, DEVICE_ID_NAMES, print_leak_stats, remove_ch_name_url_false_positives

## Load leaks
- Run the following to detect and pickle leaks
  - Detect on all crawls: `python2 detect_leaks.py`
  - Detect on a single crawl: `python2 detect_leaks.py roku-data-20190508-013650`

In [3]:
leaks_roku = load_df(CrawlRokuTop1KNoMITM, "leak")
remove_ch_name_url_false_positives(leaks_roku)  # this is part of the leak detection flow now
id_leaks_roku = leaks_roku[leaks_roku.id_type.isin(DEVICE_ID_NAMES)]
df = print_leak_stats(leaks_roku)
df

,ID,Num. of leaks,Num. of channels
0,AD ID,4606,313
1,Build Number,2910,317
2,Channel name,7291,225
3,Serial No,1708,108
4,City,30,5
5,State,11,2
6,Zip,35,5
7,Email,2,1


In [13]:
print(make_latex_table(
    df, caption="Overview of information leakage detected in Roku-Top1K-NoMITM crawl",
    label="leaks_roku"))


\begin{table}[H]
%\centering
\resizebox{\columnwidth}{!}{%
\begin{tabular}{lrr}
\toprule
 Id        &   Num. of leaks &   Num. of channels \\
\midrule
 AD ID     &            1029 &                111 \\
 Serial No &             964 &                 75 \\
\bottomrule
\end{tabular}
}
\caption{Overview of information leakage detected in Roku-Top1K-NoMITM crawl}
\label{tab:leaks_roku}
\end{table}


In [14]:
## Pi-Hole-Block

## ID Leaks
- Exclude non-Id search terms

In [15]:
id_leaks_roku.adblocked.value_counts()

True     4853
False    1461
Name: adblocked, dtype: int64

### Email sent on channel (for registration)
- email address sant to  http://api.qello.com/users/register/ for registration purposes
- crawler actually clicked the dialog to allow email address to be accessed from Roku

In [12]:
leaks_roku[leaks_roku.id_type.isin(["Email", "Unknown"])]

,adblocked,adblocked_by_url,category,channel_id,channel_name,cookie,decoded_data,disconnect_blocked,disconnect_blocked_by_url,domain_by_dns,...,referer,req_domain,search,status,tcp_dstport,tcp_stream,time,url,user_agent,ch_name_url_false_pos
0,False,False,Music,40299,Stingray Qello,,,False,False,qello.com,...,,qello.com,macyli47@gmail.com,TERMINATED,80,84,1557386763.457430000,http://api.qello.com/users/register/,Roku/DVP-9.0 (519.00E04142A),False
0,False,False,Music,40299,Stingray Qello,,,False,False,qello.com,...,,qello.com,macyli47@gmail.com,TERMINATED,80,59,1557386729.679612000,http://api.qello.com/users/register/,Roku/DVP-9.0 (519.00E04142A),False


### Adblocked status
- 4853 of the 6314 (76%) ID leaks are to domains flagged by ad blocker lists
- 38 if the 76 domains IDs leaked to are flagged by ad blocker lists

In [ ]:
id_leaks_roku.adblocked.value_counts()

In [ ]:
leaks_roku.adblocked.value_counts()

In [ ]:
id_leaks_roku.drop_duplicates("req_domain").adblocked.value_counts()

### Leaked IDs

In [ ]:
leaks_roku.id_type.value_counts()

### Leak location

In [ ]:
leaks_roku.leak_type.value_counts()

### Leak encodings

In [ ]:
leaks_roku.encoding.value_counts()